In [20]:
import torch
import numpy as np
import sounddevice as sd
from scipy.signal import resample
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline

In [21]:
# Device & dtype setup
device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(device)
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32
print(torch_dtype)

cuda:0
torch.float16


In [22]:
# Load Whisper Large V3 Turbo
model_id = r"D:\ASR_Model\whisper-large-v3-turbo"

In [23]:
model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id,
    torch_dtype=torch_dtype,
    #low_cpu_mem_usage=True,
    use_safetensors=True,
)
model.to(device)

WhisperForConditionalGeneration(
  (model): WhisperModel(
    (encoder): WhisperEncoder(
      (conv1): Conv1d(128, 1280, kernel_size=(3,), stride=(1,), padding=(1,))
      (conv2): Conv1d(1280, 1280, kernel_size=(3,), stride=(2,), padding=(1,))
      (embed_positions): Embedding(1500, 1280)
      (layers): ModuleList(
        (0-31): 32 x WhisperEncoderLayer(
          (self_attn): WhisperAttention(
            (k_proj): Linear(in_features=1280, out_features=1280, bias=False)
            (v_proj): Linear(in_features=1280, out_features=1280, bias=True)
            (q_proj): Linear(in_features=1280, out_features=1280, bias=True)
            (out_proj): Linear(in_features=1280, out_features=1280, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1280, out_features=5120, bias=True)
          (fc2): Linear(in_features=5120, out_features=1280, bias=Tr

In [24]:
processor = AutoProcessor.from_pretrained(model_id)

In [25]:
pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch_dtype,
    device=device,
)

In [34]:
# Real-time streaming function
def real_time_transcribe(duration_chunk=4.0, input_rate=44100, model_rate=16000):
    buffer = []
    print("Start speaking. Press Ctrl+C to stop.\n")

    def callback(indata, frames, time, status):
        if status:
            print("Status:", status)
        audio = indata[:, 0]  # mono channel
        resampled = resample(audio, int(len(audio) * model_rate / input_rate)).astype(np.float32)
        #result = pipe(resampled, generate_kwargs={"language": "de"})
        result = pipe(resampled, generate_kwargs={"language": "german", "task": "transcribe"})
        text = result.get("text", "").strip()
        if text:
            buffer.append(text)
            print(" ".join(buffer))

    try:
        with sd.InputStream(callback=callback,
                            channels=1,
                            samplerate=input_rate,
                            blocksize=int(duration_chunk * input_rate),
                            dtype='float32'):
            while True:
                sd.sleep(1000)
    except KeyboardInterrupt:
        print("\nTranscription stopped.")
        print("\nFinal Transcript:\n", " ".join(buffer))

# Start real-time transcription
real_time_transcribe()

Start speaking. Press Ctrl+C to stop.

Meine kurze Vorstellung. Hallo.
Meine kurze Vorstellung. Hallo. Ich heiße Maria. Ich bin 26 Jahre alt und komme aus
Meine kurze Vorstellung. Hallo. Ich heiße Maria. Ich bin 26 Jahre alt und komme aus Seit sechs Monaten lebe ich in Deutschland, in München.
Meine kurze Vorstellung. Hallo. Ich heiße Maria. Ich bin 26 Jahre alt und komme aus Seit sechs Monaten lebe ich in Deutschland, in München. Ich arbeite als Lehrerin und unterrichte Englisch.
Meine kurze Vorstellung. Hallo. Ich heiße Maria. Ich bin 26 Jahre alt und komme aus Seit sechs Monaten lebe ich in Deutschland, in München. Ich arbeite als Lehrerin und unterrichte Englisch. In meiner Freizeit gehe ich gerne spazieren und treffe mich mit Freundschaften.
Meine kurze Vorstellung. Hallo. Ich heiße Maria. Ich bin 26 Jahre alt und komme aus Seit sechs Monaten lebe ich in Deutschland, in München. Ich arbeite als Lehrerin und unterrichte Englisch. In meiner Freizeit gehe ich gerne spazieren und tref